In [18]:
repos = {'layer1':[], 'layer2':[], 'layer3':[], 'layer4':[], 'layer5':[], 'layer6':[]}

In [38]:
repos['layer7'] = []

In [39]:
import re
import asyncio
from playwright.async_api import async_playwright
import json

# Open java_repos2.json
#with open("java_repos2.json", "r") as f:
#    repos = json.load(f)

urls = {}
all_dependencies = {}

async def scrape_dependencies():
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        for repo in repos['layer6']:
            #print(f"Testing {repo['name']}...")
            print(f"Testing {repo}...")
            repo_name = repo #['full_name']
            repo_dependencies = []

            try:
                # Navigate to the repository dependencies page
                await page.goto(f"https://github.com/{repo_name}/network/dependencies", timeout=30000)
                
                # Check for "no dependencies found" or disabled dependencies messages
                try:
                    # Wait a bit for the page to load
                    await asyncio.sleep(2)
                    
                    # Check for various "no dependencies" messages
                    page_content = await page.content()
                    
                    if any(phrase in page_content.lower() for phrase in [
                        "no dependencies found",
                        "dependencies disabled",
                        "dependency graph disabled",
                        "dependency graph is disabled",
                        "no manifest files found",
                        "this repository doesn't have any dependencies"
                    ]):
                        print(f"  Skipping {repo_name}: No dependencies found or disabled")
                        all_dependencies[repo_name] = []
                        continue
                        
                    # Also check for specific UI elements that indicate no dependencies
                    no_deps_selectors = [
                        "text=No dependencies found",
                        "text=Dependencies disabled",
                        "text=Dependency graph disabled",
                        "text=Dependency graph is disabled",
                        "h3[data-view-component='true'].blankslate-heading:has-text('Dependency graph is disabled')",
                        "[data-testid='no-dependencies']"
                    ]
                    
                    for selector in no_deps_selectors:
                        try:
                            element = await page.wait_for_selector(selector, timeout=3000)
                            if element:
                                print(f"  Skipping {repo_name}: Found no dependencies indicator")
                                all_dependencies[repo_name] = []
                                break
                        except:
                            continue
                    else:
                        # If we didn't find any "no dependencies" indicators, proceed
                        
                        # Wait for the dependency boxes to load
                        await page.wait_for_selector('div[data-view-component="true"].Box', timeout=10000)

                        # Find all dependency boxes
                        box_divs = await page.query_selector_all('div[data-view-component="true"].Box')
                        
                        for box_div in box_divs:
                            # Look for dependency lists in each box
                            ul_elements = await box_div.query_selector_all("ul")
                            
                            for ul_element in ul_elements:
                                # Find all dependency links in this list
                                a_tags = await ul_element.query_selector_all('a.h4.Link--primary.Link')
                                
                                for a_tag in a_tags:
                                    try:
                                        # Get the dependency URL
                                        url = await a_tag.get_attribute('href')
                                        
                                        # Get the dependency name text
                                        dep_text = await a_tag.text_content()
                                        
                                        if url and dep_text:
                                            dependency_info = {
                                                'name': dep_text.strip(),
                                                'url': url,
                                                'full_url': f"https://github.com{url}" if url.startswith('/') else url
                                            }
                                            repo_dependencies.append(dependency_info)
                                            print(f"  Found dependency: {dep_text.strip()} -> {url}")
                                    except Exception as e:
                                        print(f"  Error extracting dependency info: {e}")
                                        continue

                        # Store dependencies for this repo
                        all_dependencies[repo_name] = repo_dependencies
                        
                except Exception as e:
                    print(f"  Error checking for dependencies: {e}")
                    all_dependencies[repo_name] = []
                    continue
                
                # Also store URLs in the original format for backward compatibility
                if repo_dependencies:
                    urls[repo_name] = [dep['url'] for dep in repo_dependencies]
                
                print(f"  Total dependencies found for {repo_name}: {len(repo_dependencies)}")
                
                # Add a small delay to be respectful
                await asyncio.sleep(1)
                
            except Exception as e:
                print(f"  Error processing {repo_name}: {e}")
                all_dependencies[repo_name] = []
                continue

        await context.close()
        await browser.close()

# Run the async function
await scrape_dependencies()

# Save results
with open("scraped_dependencies.json", "w") as f:
    json.dump({
        'dependencies': all_dependencies,
        'urls': urls,
        'summary': {
            'total_repos_processed': len(all_dependencies),
            'repos_with_dependencies': len([r for r in all_dependencies.values() if r]),
            'total_dependencies_found': sum(len(deps) for deps in all_dependencies.values())
        }
    }, f, indent=2)

print(f"\nScraping completed!")
print(f"Total repositories processed: {len(all_dependencies)}")
print(f"Repositories with dependencies: {len([r for r in all_dependencies.values() if r])}")
print(f"Total dependencies found: {sum(len(deps) for deps in all_dependencies.values())}")
print(f"Results saved to scraped_dependencies.json")

Testing akoufoudakis/jsr305...
  Found dependency: org.apache.felix:maven-bundle-plugin -> /sonatype/sonatype-bundle-plugin
  Found dependency: org.apache.maven.plugins:maven-assembly-plugin -> /apache/maven-plugins
  Found dependency: org.apache.maven.plugins:maven-compiler-plugin -> /apache/maven-compiler-plugin
  Found dependency: org.apache.maven.plugins:maven-enforcer-plugin -> /apache/maven-plugins
  Found dependency: org.apache.maven.plugins:maven-jar-plugin -> /apache/maven-plugins
  Found dependency: org.apache.maven.plugins:maven-javadoc-plugin -> /apache/maven-javadoc-plugin
  Found dependency: org.apache.maven.plugins:maven-source-plugin -> /apache/maven-plugins
  Found dependency: org.codehaus.mojo:buildnumber-maven-plugin -> /mojohaus/buildnumber-maven-plugin
  Total dependencies found for akoufoudakis/jsr305: 8
Testing junit-team/junit4...
  Found dependency: com.google.code.maven-replacer-plugin:replacer -> /beiliubei/maven-replacer-plugin
  Found dependency: org.apache

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("text=Dependencies disabled") to be visible\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("text=Dependencies disabled") to be visible



  Error checking for dependencies: Page.wait_for_selector: Target page, context or browser has been closed
Testing alexruiz/fest-reflect...
  Error processing alexruiz/fest-reflect: Page.goto: Target page, context or browser has been closed


TargetClosedError: BrowserContext.close: Target page, context or browser has been closed

In [40]:
len(all_dependencies)
all_dependencies

# save all_dependencies to a file
with open("layer7_dep.json", "w") as f:
    json.dump(all_dependencies, f, indent=2)

In [41]:
# Populate repos['layer2'] with all dependency URLs from all_dependencies
#with open("all_dependencies_layer2.json", "r") as f:
#    all_dependencies = json.load(f)

repos['layer7'] = []

for repo, deps in all_dependencies.items():
    for dep in deps:
        if 'url' in dep:
            repos['layer7'].append(dep['url'][1:])

print(f"Total URLs in repos['layer2']: {len(repos['layer7'])}")

Total URLs in repos['layer2']: 139


In [24]:
repos['layer2']

['alibaba/druid',
 'alibaba/fastjson2',
 'pagehelper/Mybatis-PageHelper',
 'google/guava',
 'apache/commons-codec',
 'mysql/mysql-connector-j',
 'apache/commons-lang',
 'freemarker/freemarker-old',
 'liudongdong0909/mybatis-generator-core-fix',
 'mybatis/mybatis-3',
 'mybatis/spring',
 'spring-projects/spring-boot',
 'spring-projects/spring-boot',
 'spring-projects/spring-boot',
 'spring-projects/spring-boot',
 'abel533/Mapper',
 'alibaba/druid',
 'apache/commons-io',
 'junit-team/junit4',
 'mysql/mysql-connector-j',
 'apache/curator',
 'apache/logging-log4j2',
 'apache/logging-log4j2',
 'apache/logging-log4j2',
 'apache/maven-compiler-plugin',
 'jacoco/jacoco',
 'mybatis/mybatis-3',
 'mybatis/spring',
 'codehaus/perf4j',
 'qos-ch/slf4j',
 'spring-projects/spring-framework',
 'spring-projects/spring-framework',
 'spring-projects/spring-framework',
 'spring-projects/spring-framework',
 'spring-projects/spring-framework',
 'spring-projects/spring-boot',
 'spring-projects/spring-boot',
 '

In [45]:
import glob
import json

layer_dep_files = sorted(glob.glob("layer*_dep.json"))
layers = {}

for file in layer_dep_files:
    with open(file, "r") as f:
        layer_name = file.split("_")[0]  # e.g., "layer7"
        layers[layer_name] = json.load(f)

with open("layer1.json", "r") as f:
    layer1_data = json.load(f)


clean_full_names = set()
for repo in layer1_data['layer1']:
    clean_full_names.add(repo['full_name'])

for layer in layers:
    for repo in layers[layer]:
       for dep in layers[layer][repo]:
           clean_full_names.add(dep['url'][1:])

# Save the cleaned full names to a file
with open("all_repos.json", "w") as f:
    json.dump(list(clean_full_names), f, indent=2)

In [44]:
len(clean_full_names)

1911